In [31]:
import pandas as pd
import pymysql

# Restore cleaned dataframe
%store -r df_cleaned  

# Ensure df_cleaned is loaded
if 'df_cleaned' not in locals():
    raise ValueError("DataFrame 'df_cleaned' not found. Ensure data_cleaning notebook was executed.")

# Import RDS credentials
from config import RDS_HOST, RDS_USER, RDS_PASSWORD, RDS_DATABASE

# ✅ Establish Database Connection
try:
    mydb = pymysql.connect(
        host=RDS_HOST,
        user=RDS_USER,
        password=RDS_PASSWORD,
        database=RDS_DATABASE,
        cursorclass=pymysql.cursors.DictCursor,
        autocommit=False  # Enable transactions
    )
    mycursor = mydb.cursor()  # Create cursor
    print("✅ Database connection established!")
except pymysql.MySQLError as e:
    raise RuntimeError(f"❌ Database connection failed: {e}")

# ✅ Insert Data into Item Table
for index, row in df_cleaned.iterrows():
    item_sql = """INSERT INTO Item (item_id, description, item_type, unit, location)
                   VALUES (%s, %s, %s, %s, %s)
                   ON DUPLICATE KEY UPDATE description=%s, item_type=%s, unit=%s, location=%s"""
    
    item_values = (
        row['item_id'], row.get('description', 'Unknown'), row.get('item_type', 'Unknown'),
        row.get('unit', 'Unknown'), row.get('location', 'Unknown'),
        row.get('description', 'Unknown'), row.get('item_type', 'Unknown'),
        row.get('unit', 'Unknown'), row.get('location', 'Unknown')
    )

    try:
        mycursor.execute(item_sql, item_values)
    except pymysql.MySQLError as e:
        print(f"❌ Error inserting/updating Item: {e}. Values: {item_values}")
        mydb.rollback()
        raise

# ✅ Insert Data into Inventory Table
for index, row in df_cleaned.iterrows():
    inventory_sql = """INSERT INTO Inventory (item_id, quantity_on_hand, cost, purchase_date, vendor)
                       VALUES (%s, %s, %s, %s, %s)"""
    
    inventory_values = [row.get(col, None) for col in ['item_id', 'quantity_on_hand', 'cost', 'purchase_date', 'vendor']]

    try:
        mycursor.execute(inventory_sql, inventory_values)
    except pymysql.MySQLError as e:
        print(f"❌ Error inserting into Inventory: {e}. Values: {inventory_values}")
        mydb.rollback()
        raise

# ✅ Insert Data into Sales Table
for index, row in df_cleaned.iterrows():
    sales_sql = """INSERT INTO Sales (item_id, price, date_sold, cust, quantity_sold)
                   VALUES (%s, %s, %s, %s, %s)"""
    
    sales_values = [row.get(col, None) for col in ['item_id', 'price', 'date_sold', 'cust', 'quantity_sold']]

    try:
        mycursor.execute(sales_sql, sales_values)
    except pymysql.MySQLError as e:
        print(f"❌ Error inserting into Sales: {e}. Values: {sales_values}")
        mydb.rollback()
        raise

# ✅ Commit the changes to save data
mydb.commit()
print("🎉 Data loaded successfully!")

# ✅ Close connection
if mydb.open:
    mycursor.close()
    mydb.close()
    print("🔌 Database connection closed.")


✅ Database connection established!
🎉 Data loaded successfully!
🔌 Database connection closed.
